In [1]:
from neo4j import GraphDatabase
 
# Define correct URI and AUTH arguments (no AUTH by default)
URI = "bolt://localhost:7687"
AUTH = ("", "")
 
with GraphDatabase.driver(URI, auth=AUTH) as client:
    # Check the connection
    client.verify_connectivity()
 
    # Find a user John in the database
    records, summary, keys = client.execute_query(
        "CALL export_util.json('/usr/lib/memgraph/query_modules/export.json') YIELD objects return objects;",
        database_="memgraph",
    )
 
    # Get the result
    for record in records:
        print(record["name"])
 
    # Print the query
    print(summary.query)

ClientError: {code: Memgraph.ClientError.MemgraphError.MemgraphError} {message: export_util.json: Traceback (most recent call last):
  File "/usr/lib/memgraph/query_modules/export_util.py", line 542, in json_dump_to_file
    with open(path, "w") as outfile:
PermissionError: [Errno 13] Permission denied: '/usr/lib/memgraph/query_modules/export.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/memgraph/query_modules/export_util.py", line 583, in json
    json_dump_to_file(graph, path)
  File "/usr/lib/memgraph/query_modules/export_util.py", line 550, in json_dump_to_file
    raise PermissionError(
PermissionError: You don't have permissions to write into that file. Make sure to give the necessary permissions to user memgraph.
}

In [68]:
def restructure_json_list(data_list):
    """
    Restructures a list of JSON-like dictionaries to group relationships by the 'relationship' key.

    Args:
        data_list: A list of dictionaries, where each dictionary contains a 'relationship' key.

    Returns:
        A dictionary where the 'relationship' key maps to a list of relationship dictionaries.
    """
    if not isinstance(data_list, list):
      return data_list #raise TypeError("Input must be a list")

    nodes = []
    for item in data_list:
        if isinstance(item, dict) and 'node' in item and isinstance(item['node'], dict):
            node = item['node'].copy()  # Create a copy to avoid modifying the original
            if 'name' in node:
                node['id'] = node.pop('name')  # Rename 'name' to 'id'
            nodes.append(node)

    #return {"nodes": nodes}

    relationships = []
    for item in data_list:
        if isinstance(item, dict) and 'relationship' in item and isinstance(item['relationship'], dict):
            relationships.append(item['relationship'])
        #if you have other keys in the original list, you can add more elif statements here.

    return {"nodes": nodes,
        "relationships": relationships}

In [83]:
def transform_nodes(nodes_list):
    """
    Transforms a list of node dictionaries from the given format to {'id': '...', 'type': '...'}.

    Args:
        nodes_list: A list of node dictionaries.

    Returns:
        A list of transformed node dictionaries.
    """
    if not isinstance(nodes_list, list):
      raise TypeError("Input must be a list")

    transformed_nodes = []
    for node in nodes_list:
        if isinstance(node, dict) and 'id' in node and 'labels' in node and isinstance(node['labels'], list) and len(node['labels']) > 0:
            transformed_node = {'id': node['id'], 'type': node['labels'][0].replace(" ", "")} #remove spaces in the type name.
            transformed_nodes.append(transformed_node)
        #if you have other node formats, add elif statements here.
        else:
            transformed_nodes.append(node)
    return transformed_nodes

In [85]:
from neo4j import GraphDatabase
import json

# Define connection details
URI = "bolt://localhost:7687"
AUTH = ("", "")  # No authentication for Memgraph

# Load JSON data from a file
def load_json_from_file(filename):
    with open(filename, "r", encoding="utf-8") as file:
        print("Opening file: ", filename)
        return json.load(file)

# Function to load data into Memgraph
def load_graph_from_json(json_data):
    with GraphDatabase.driver(URI, auth=AUTH) as client:
        client.verify_connectivity()
        json_data = restructure_json_list(json_data)

        

        # Create Nodes
        for node in transform_nodes(json_data["nodes"]):
            query = """
            MERGE (n:$type {id: $id})
            RETURN n.id AS name;
            """
            
            records, _, _ = client.execute_query(query, type=node["type"], id=node["id"], database_="memgraph")

            # Print created nodes
            for record in records:
                print("Created node:", record["name"])

        # Create Relationships
        for rel in transform_nodes(json_data["relationships"]):
            source = rel["source"]
            target = rel["target"]
            relation = rel["relation"]

            query = f"""
            MATCH (s {{id: $source}}), (t {{id: $target}})
            MERGE (s)-[r:{relation}]->(t)
            RETURN type(r) AS relation;
            """
            
            print("Executing Query:", query)  # Debugging output
            
            records, _, _ = client.execute_query(
                query, source=source, target=target, database_="memgraph"
            )

            # Print created relationships
            for record in records:
                print("Created relationship:", record["relation"])

# Load JSON from file and insert it into Memgraph
#json_filename = "answer_from_LLM.json"  # Replace with your actual file path
#json_data = load_json_from_file(json_filename)
#load_graph_from_json(json_data)


In [2]:
# UUS 14.04

from neo4j import GraphDatabase
import json

# Define connection details
URI = "bolt://localhost:7687"
AUTH = ("", "")  # No authentication for Memgraph

# Load JSON data from a file
def load_json_from_file(filename):
    with open(filename, "r", encoding="utf-8") as file:
        print("Opening file: ", filename)
        return json.load(file)

def parse_identifier(identifier):
    """Parses 'schema.table.column' into (schema, table, column)."""
    parts = identifier.split('.')
    if len(parts) == 3:
        return parts[0], parts[1], parts[2]
    elif len(parts) == 2: # Handle case like 'schema.table' if needed, though column expected
         print(f"Warning: Identifier '{identifier}' only has 2 parts. Assuming table level.")
         return parts[0], parts[1], None
    else:
        raise ValueError(f"Invalid identifier format: {identifier}")


# Function to load data into Memgraph
def load_graph_from_json(json_data):
    with GraphDatabase.driver(URI, auth=AUTH) as client:
        client.verify_connectivity()

def load_lineage_to_memgraph(db, data):
    """Loads the lineage JSON data into Memgraph."""
    target_full_table_name = data['target_table']
    target_schema_name, target_table_name = target_full_table_name.split('.')




lineage_path = "C:\lopu-kg-test\project\src\LLM_answers\llm_prompt_for_column_level_lineage_hard_wo_ex\\answer_3_wh_db.DimCustomer_20250411_180418.json"  # -- NENDEGA PRAEGU EI TÖÖTA, sest on liiga detailsed
lineage_content = load_json_from_file(lineage_path)

Opening file:  C:\lopu-kg-test\project\src\LLM_answers\llm_prompt_for_column_level_lineage_hard_wo_ex\answer_3_wh_db.DimCustomer_20250411_180418.json


In [3]:
def transform_lineage_to_nodes_rels(data):
    """
    Transforms the original lineage JSON into separate lists of nodes
    and relationships suitable for the generic loading pattern.
    """
    nodes = {} # Use dict to ensure uniqueness by full_name
    relationships = []

    target_full_table_name = data['target_table']
    target_schema_name, target_table_name = target_full_table_name.split('.')

    # Add target schema, table
    nodes[target_schema_name] = {"id": target_schema_name, "label": "Schema", "props": {"name": target_schema_name}}
    nodes[target_full_table_name] = {"id": target_full_table_name, "label": "Table", "props": {"name": target_table_name, "full_name": target_full_table_name}}
    relationships.append({"source": target_full_table_name, "target": target_schema_name, "type": "BELONGS_TO", "props": {}})

    # Process each target column
    for target_col_name, lineage_info in data['lineage'].items():
        target_col_full_name = f"{target_full_table_name}.{target_col_name}"

        # Add target column node
        nodes[target_col_full_name] = {"id": target_col_full_name, "label": "Column", "props": {"name": target_col_name, "full_name": target_col_full_name}}
        relationships.append({"source": target_full_table_name, "target": target_col_full_name, "type": "HAS_COLUMN", "props": {}})

        # Process sources
        for source_info in lineage_info['sources']:
            try:
                source_identifier = source_info['source_identifier']
                src_schema_name, src_table_name, src_col_name = parse_identifier(source_identifier)
                src_full_table_name = f"{src_schema_name}.{src_table_name}"
                src_col_full_name = f"{src_full_table_name}.{src_col_name}"

                # Add source schema, table, column nodes (if not already added)
                if src_schema_name not in nodes:
                    nodes[src_schema_name] = {"id": src_schema_name, "label": "Schema", "props": {"name": src_schema_name}}
                if src_full_table_name not in nodes:
                    nodes[src_full_table_name] = {"id": src_full_table_name, "label": "Table", "props": {"name": src_table_name, "full_name": src_full_table_name}}
                    relationships.append({"source": src_full_table_name, "target": src_schema_name, "type": "BELONGS_TO", "props": {}})
                if src_col_full_name not in nodes:
                    nodes[src_col_full_name] = {"id": src_col_full_name, "label": "Column", "props": {"name": src_col_name, "full_name": src_col_full_name}}
                    relationships.append({"source": src_full_table_name, "target": src_col_full_name, "type": "HAS_COLUMN", "props": {}})

                # Prepare DERIVES relationship properties
                rel_props = {
                    "transformation_type": lineage_info.get("transformation_type"),
                    "transformation_logic": lineage_info.get("transformation_logic"),
                    "path": str(source_info.get("path")),
                    "role": source_info.get("role"),
                    "join_info": json.dumps(source_info.get("join_info")) if source_info.get("join_info") else None,
                    "notes": lineage_info.get("notes"),
                }
                rel_props = {k: v for k, v in rel_props.items() if v is not None} # Clean None values

                # Add DERIVES relationship
                relationships.append({
                    "source": src_col_full_name,
                    "target": target_col_full_name,
                    "type": "DERIVES",
                    "props": rel_props
                })

            except ValueError as e:
                print(f"    Skipping source due to error: {e}")
            except Exception as e:
                 print(f"    An unexpected error occurred processing source {source_info.get('source_identifier', 'UNKNOWN')}: {e}")

    return {"nodes": list(nodes.values()), "relationships": relationships}

transform_lineage_to_nodes_rels(lineage_content)

    An unexpected error occurred processing source UNKNOWN: 'source_identifier'
    An unexpected error occurred processing source UNKNOWN: 'source_identifier'
    An unexpected error occurred processing source UNKNOWN: 'source_identifier'
    An unexpected error occurred processing source UNKNOWN: 'source_identifier'
    An unexpected error occurred processing source UNKNOWN: 'source_identifier'
    An unexpected error occurred processing source UNKNOWN: 'source_identifier'
    An unexpected error occurred processing source UNKNOWN: 'source_identifier'
    An unexpected error occurred processing source UNKNOWN: 'source_identifier'
    An unexpected error occurred processing source UNKNOWN: 'source_identifier'
    An unexpected error occurred processing source UNKNOWN: 'source_identifier'
    An unexpected error occurred processing source UNKNOWN: 'source_identifier'
    An unexpected error occurred processing source UNKNOWN: 'source_identifier'
    An unexpected error occurred process

{'nodes': [{'id': 'wh_db', 'label': 'Schema', 'props': {'name': 'wh_db'}},
  {'id': 'wh_db.DimCustomer',
   'label': 'Table',
   'props': {'name': 'DimCustomer', 'full_name': 'wh_db.DimCustomer'}},
  {'id': 'wh_db.DimCustomer.sk_customerid',
   'label': 'Column',
   'props': {'name': 'sk_customerid',
    'full_name': 'wh_db.DimCustomer.sk_customerid'}},
  {'id': 'wh_db.DimCustomer.customerid',
   'label': 'Column',
   'props': {'name': 'customerid',
    'full_name': 'wh_db.DimCustomer.customerid'}},
  {'id': 'wh_db.DimCustomer.taxid',
   'label': 'Column',
   'props': {'name': 'taxid', 'full_name': 'wh_db.DimCustomer.taxid'}},
  {'id': 'wh_db.DimCustomer.status',
   'label': 'Column',
   'props': {'name': 'status', 'full_name': 'wh_db.DimCustomer.status'}},
  {'id': 'wh_db.DimCustomer.lastname',
   'label': 'Column',
   'props': {'name': 'lastname', 'full_name': 'wh_db.DimCustomer.lastname'}},
  {'id': 'wh_db.DimCustomer.firstname',
   'label': 'Column',
   'props': {'name': 'firstname

In [ ]:
import json
from neo4j import GraphDatabase, basic_auth # Import Neo4j driver components

# --- Configuration ---
# NOTE: Use 'neo4j://...' or 'bolt://...' for the URI with the neo4j driver
# Memgraph typically runs Bolt on 7687
URI = "bolt://localhost:7687"
# If using Memgraph Cloud / AuraDB or Memgraph with authentication enabled:
# AUTH = basic_auth("your_user", "your_password")
AUTH = None # Set to None or remove if no authentication

# Your lineage data (same as before)
lineage_data = {
  "target_table": "wh.DimComp",
  # ... (rest of your lineage_data JSON) ...
  "lineage": {
    "sk_compid": {
      "sources": [
        {"source_identifier": "wh_stg.FW.value", "path": ["cmp.RecDate", "T.effectivedate"], "role": "Date part source"},
        {"source_identifier": "wh_stg.FW.value", "path": ["cmp.CIK", "T.compid"], "role": "ID part source"}
      ],
      "transformation_type": "EXPRESSION_CONCAT",
      "transformation_logic": "CAST(strftime(T.effectivedate, '%Y%m%d') || T.compid AS BIGINT)"
    },
    "compid": {
      "sources": [{"source_identifier": "wh_stg.FW.value", "path": ["cmp.CIK"]}],
      "transformation_type": "CAST",
      "transformation_logic": "CAST(cmp.CIK AS BIGINT)"
    },
    "status": {
      "sources": [{"source_identifier": "wh_stg.FW.value", "path": ["cmp.Status"]}],
      "transformation_type": "CASE_MAPPING",
      "transformation_logic": "CASE cmp.Status WHEN 'ACTV' THEN 'Active' ELSE 'Inactive' END"
    },
    "name": {
      "sources": [{"source_identifier": "wh_stg.FW.value", "path": ["cmp.CompName"]}],
      "transformation_type": "RENAME",
      "transformation_logic": "cmp.CompName AS name"
    },
    "industry": {
      "sources": [
          {"source_identifier": "wh.Ind.in_name",
           "join_info": {
             "type": "INNER JOIN",
             "left_source": {"identifier": "wh_stg.FW.value", "path": ["cmp.IndID"]},
             "right_source": {"identifier": "wh.Ind.in_id"}
           }
          }
      ],
      "transformation_type": "JOIN_LOOKUP",
      "transformation_logic": "ind.in_name via JOIN ON cmp.IndID = ind.in_id"
    },
    "effectivedate": {
        "sources": [{"source_identifier": "wh_stg.FW.value", "path": ["cmp.RecDate"]}],
        "transformation_type": "RENAME",
        "transformation_logic": "cmp.RecDate AS effectivedate"
    },
    "enddate": {
      "sources": [
        {"source_identifier": "wh_stg.FW.value", "path": ["cmp.RecDate"], "role": "Input for LEAD"},
        {"source_identifier": "wh_stg.FW.value", "path": ["cmp.CIK"], "role": "Partition key for LEAD"}
      ],
      "transformation_type": "WINDOW_FUNCTION",
      "transformation_logic": "COALESCE(LEAD(cmp.RecDate) OVER (PARTITION BY cmp.CIK ORDER BY cmp.RecDate), '9999-12-31'::DATE)",
      "notes": "SCD2 end date calculation"
    }
  }
}


# --- Helper Function ---
def parse_identifier(identifier):
    parts = identifier.split('.')
    if len(parts) == 3:
        return parts[0], parts[1], parts[2]
    elif len(parts) == 2:
         print(f"Warning: Identifier '{identifier}' only has 2 parts. Assuming table level.")
         return parts[0], parts[1], None
    else:
        raise ValueError(f"Invalid identifier format: {identifier}")

# --- Data Transformation Step (Required for the neo4j driver template) ---
def transform_lineage_to_nodes_rels(data):
    """
    Transforms the original lineage JSON into separate lists of nodes
    and relationships suitable for the generic loading pattern.
    """
    nodes = {} # Use dict to ensure uniqueness by full_name
    relationships = []

    target_full_table_name = data['target_table']
    target_schema_name, target_table_name = target_full_table_name.split('.')

    # Add target schema, table
    nodes[target_schema_name] = {"id": target_schema_name, "label": "Schema", "props": {"name": target_schema_name}}
    nodes[target_full_table_name] = {"id": target_full_table_name, "label": "Table", "props": {"name": target_table_name, "full_name": target_full_table_name}}
    relationships.append({"source": target_full_table_name, "target": target_schema_name, "type": "BELONGS_TO", "props": {}})

    # Process each target column
    for target_col_name, lineage_info in data['lineage'].items():
        target_col_full_name = f"{target_full_table_name}.{target_col_name}"

        # Add target column node
        nodes[target_col_full_name] = {"id": target_col_full_name, "label": "Column", "props": {"name": target_col_name, "full_name": target_col_full_name}}
        relationships.append({"source": target_full_table_name, "target": target_col_full_name, "type": "HAS_COLUMN", "props": {}})

        # Process sources
        for source_info in lineage_info['sources']:
            try:
                source_identifier = source_info['source_identifier']
                src_schema_name, src_table_name, src_col_name = parse_identifier(source_identifier)
                src_full_table_name = f"{src_schema_name}.{src_table_name}"
                src_col_full_name = f"{src_full_table_name}.{src_col_name}"

                # Add source schema, table, column nodes (if not already added)
                if src_schema_name not in nodes:
                    nodes[src_schema_name] = {"id": src_schema_name, "label": "Schema", "props": {"name": src_schema_name}}
                if src_full_table_name not in nodes:
                    nodes[src_full_table_name] = {"id": src_full_table_name, "label": "Table", "props": {"name": src_table_name, "full_name": src_full_table_name}}
                    relationships.append({"source": src_full_table_name, "target": src_schema_name, "type": "BELONGS_TO", "props": {}})
                if src_col_full_name not in nodes:
                    nodes[src_col_full_name] = {"id": src_col_full_name, "label": "Column", "props": {"name": src_col_name, "full_name": src_col_full_name}}
                    relationships.append({"source": src_full_table_name, "target": src_col_full_name, "type": "HAS_COLUMN", "props": {}})

                # Prepare DERIVES relationship properties
                rel_props = {
                    "transformation_type": lineage_info.get("transformation_type"),
                    "transformation_logic": lineage_info.get("transformation_logic"),
                    "path": str(source_info.get("path")),
                    "role": source_info.get("role"),
                    "join_info": json.dumps(source_info.get("join_info")) if source_info.get("join_info") else None,
                    "notes": lineage_info.get("notes"),
                }
                rel_props = {k: v for k, v in rel_props.items() if v is not None} # Clean None values

                # Add DERIVES relationship
                relationships.append({
                    "source": src_col_full_name,
                    "target": target_col_full_name,
                    "type": "DERIVES",
                    "props": rel_props
                })

            except ValueError as e:
                print(f"    Skipping source due to error: {e}")
            except Exception as e:
                 print(f"    An unexpected error occurred processing source {source_info.get('source_identifier', 'UNKNOWN')}: {e}")

    return {"nodes": list(nodes.values()), "relationships": relationships}

# --- Main Loading Logic using neo4j driver ---
def load_graph_with_neo4j_driver(data_for_graph):
    """Loads nodes and relationships using the neo4j driver."""
    # Note: Specifying database_="memgraph" might not be necessary unless
    # your Memgraph instance explicitly requires multi-database support AND
    # the driver/Memgraph version supports it via Bolt. Often, you connect
    # directly to the target database implicitly. Remove if it causes issues.
    database_name = "memgraph" # Or None if not needed/supported

    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("Connected to Memgraph using neo4j driver.")

        # Optional: Create constraints/indexes (run once or ensure idempotency)
        try:
            constraints_indexes = [
                "CREATE CONSTRAINT IF NOT EXISTS FOR (s:Schema) REQUIRE s.name IS UNIQUE;",
                "CREATE INDEX IF NOT EXISTS FOR (s:Schema) ON (s.name);",
                "CREATE CONSTRAINT IF NOT EXISTS FOR (t:Table) REQUIRE t.full_name IS UNIQUE;",
                "CREATE INDEX IF NOT EXISTS FOR (t:Table) ON (t.full_name);",
                "CREATE CONSTRAINT IF NOT EXISTS FOR (c:Column) REQUIRE c.full_name IS UNIQUE;",
                "CREATE INDEX IF NOT EXISTS FOR (c:Column) ON (c.full_name);",
            ]
            for query in constraints_indexes:
                 # Use session for auto-commit or explicit transaction
                 driver.execute_query(query, database_=database_name)
            print("Ensured necessary constraints and indexes exist.")
        except Exception as e:
            print(f"Warning: Could not ensure constraints/indexes (might exist already): {e}")


        # Create/Merge Nodes
        print("\n--- Merging Nodes ---")
        for node_data in data_for_graph["nodes"]:
            # Use explicit labels and the correct identifier (full_name or name)
            # MERGE based on the unique identifier property
            identifier_key = "full_name" if node_data["label"] in ["Table", "Column"] else "name"
            identifier_value = node_data["props"][identifier_key]

            query = f"""
            MERGE (n:{node_data['label']} {{{identifier_key}: $id_val}})
            ON CREATE SET n = $props
            ON MATCH SET n += $props // Update properties if node exists
            RETURN n.{identifier_key} AS id
            """
            try:
                summary = driver.execute_query(
                    query,
                    id_val=identifier_value,
                    props=node_data["props"],
                    database_=database_name
                )
                # Access results correctly if needed (summary often just confirms execution)
                # print(f"  Merged node: {node_data['label']} {identifier_value}")
            except Exception as e:
                print(f"  Error merging node {node_data['label']} {identifier_value}: {e}")
                print(f"    Query: {query}")
                print(f"    Params: id_val={identifier_value}, props={node_data['props']}")


        # Create/Merge Relationships
        print("\n--- Merging Relationships ---")
        for rel_data in data_for_graph["relationships"]:
            source_id = rel_data["source"]
            target_id = rel_data["target"]
            rel_type = rel_data["type"]
            rel_props = rel_data["props"]

            # Determine match keys for source/target nodes
            # This assumes nodes were created/merged using specific unique keys
            # We need to know the *label* to know which key to match on! This is awkward.
            # Finding the label requires looking back at the node list or a more complex query.
            # --> Sticking to full_name/name as the assumed key for MERGE is simpler here.
            # --> This highlights a weakness of the generic node/rel structure compared to specific queries.

            # Simplified: Assume Table/Column use full_name, Schema uses name.
            # A better implementation might pass the label info along.
            source_key = "full_name" if "." in source_id else "name"
            target_key = "full_name" if "." in target_id else "name"


            # Use parameters for properties, but f-string/concatenation for relationship type
            query = f"""
            MATCH (s {{{source_key}: $source_val}})
            MATCH (t {{{target_key}: $target_val}})
            MERGE (s)-[r:{rel_type}]->(t)
            ON CREATE SET r = $props
            ON MATCH SET r += $props // Update properties
            RETURN type(r) AS type
            """
            try:
                summary = driver.execute_query(
                    query,
                    source_val=source_id,
                    target_val=target_id,
                    props=rel_props,
                    database_=database_name
                )
                # print(f"  Merged relationship: ({source_id})-[{rel_type}]->({target_id})")
            except Exception as e:
                 print(f"  Error merging relationship ({source_id})-[{rel_type}]->({target_id}): {e}")
                 print(f"    Query: {query}")
                 print(f"    Params: source_val={source_id}, target_val={target_id}, props={rel_props}")


# --- Run the process ---
try:
    # 1. Transform the original data
    print("Transforming lineage data...")
    graph_data = transform_lineage_to_nodes_rels(lineage_data)
    # print("\nTransformed Data:")
    # print(json.dumps(graph_data, indent=2)) # Optional: view transformed data

    # 2. Load the transformed data using neo4j driver
    load_graph_with_neo4j_driver(graph_data)

    print("\nLineage loading process complete using neo4j driver.")

except Exception as e:
    print(f"\nAn overall error occurred: {e}")

In [86]:
import os

llm_answers_path = r"LLM_answers"
json_files = [f for f in os.listdir(llm_answers_path) if f.endswith(".json")]


for filename in json_files:
    llm_json_path = os.path.join(llm_answers_path, filename)
    json_data = load_json_from_file(llm_json_path)
    load_graph_from_json(json_data)

    

Opening file:  LLM_answers\answer_20250305_022711.json
Created node: ProspectIncremental
Created node: wh_db_stage
Created node: agencyid
Created node: lastname
Created node: firstname
Created node: middleinitial
Created node: gender
Created node: addressline1
Created node: addressline2
Created node: postalcode
Created node: city
Created node: state
Created node: country
Created node: phone
Created node: income
Created node: numbercars
Created node: numberchildren
Created node: maritalstatus
Created node: age
Created node: creditrating
Created node: ownorrentflag
Created node: employer
Created node: numbercreditcards
Created node: networth
Created node: marketingnameplate
Created node: recordbatchid
Created node: Unique identifier from agency
Created node: Last name
Created node: First name
Created node: Middle initial
Created node: â€˜Mâ€™ or â€˜Fâ€™ or â€˜Uâ€™
Created node: Postal address
Created node: Postal address
Created node: Postal code
Created node: City
Created node: State or

In [71]:
asd = load_json_from_file(r"LLM_answers\answer_20250305_022936.json")
asd2 = load_json_from_file(r"LLM_answers\answer_20250305_022947.json")

Opening file:  LLM_answers\answer_20250305_022936.json
Opening file:  LLM_answers\answer_20250305_022947.json


In [73]:
asd["nodes"]

[{'id': 'wh_db.FactWatches',
  'labels': ['TableName'],
  'properties': {'name': 'wh_db.FactWatches'}},
 {'id': 'sk_customerid',
  'labels': ['ColumnName'],
  'properties': {'name': 'sk_customerid'}},
 {'id': 'sk_securityid',
  'labels': ['ColumnName'],
  'properties': {'name': 'sk_securityid'}},
 {'id': 'sk_dateid_dateplaced',
  'labels': ['ColumnName'],
  'properties': {'name': 'sk_dateid_dateplaced'}},
 {'id': 'sk_dateid_dateremoved',
  'labels': ['ColumnName'],
  'properties': {'name': 'sk_dateid_dateremoved'}},
 {'id': 'batchid',
  'labels': ['ColumnName'],
  'properties': {'name': 'batchid'}},
 {'id': 'BIGINT', 'labels': ['Data Type'], 'properties': {'name': 'BIGINT'}},
 {'id': 'INT', 'labels': ['Data Type'], 'properties': {'name': 'INT'}},
 {'id': 'wh_db', 'labels': ['SchemaName'], 'properties': {'name': 'wh_db'}},
 {'id': 'Customer associated with watch list',
  'labels': ['Comment'],
  'properties': {'text': 'Customer associated with watch list'}},
 {'id': 'Security listed on 

In [79]:
def transform_nodes(nodes_list):
    """
    Transforms a list of node dictionaries from the given format to {'id': '...', 'type': '...'}.

    Args:
        nodes_list: A list of node dictionaries.

    Returns:
        A list of transformed node dictionaries.
    """
    if not isinstance(nodes_list, list):
      raise TypeError("Input must be a list")

    transformed_nodes = []
    for node in nodes_list:
        if isinstance(node, dict) and 'id' in node and 'labels' in node and isinstance(node['labels'], list) and len(node['labels']) > 0:
            transformed_node = {'id': node['id'], 'type': node['labels'][0].replace(" ", "")} #remove spaces in the type name.
            transformed_nodes.append(transformed_node)
        #if you have other node formats, add elif statements here.
        else:
            transformed_nodes.append(node)
    return transformed_nodes

In [81]:
transform_nodes(asd2["nodes"])

[{'id': 'sk_companyid', 'type': 'ColumnName'},
 {'id': 'Financial', 'type': 'TableName'},
 {'id': 'BIGINT', 'type': 'Data Type'},
 {'id': 'wh_db', 'type': 'SchemaName'},
 {'id': 'Company SK.', 'type': 'Comment'},
 {'id': 'fi_year', 'type': 'ColumnName'},
 {'id': 'INT', 'type': 'Data Type'},
 {'id': 'Year of the quarter end.', 'type': 'Comment'},
 {'id': 'fi_qtr', 'type': 'ColumnName'},
 {'id': 'Quarter number that the financial information is for: valid values 1, 2, 3, 4.',
  'type': 'Comment'},
 {'id': 'fi_qtr_start_date', 'type': 'ColumnName'},
 {'id': 'DATE', 'type': 'Data Type'},
 {'id': 'Start date of quarter.', 'type': 'Comment'},
 {'id': 'fi_revenue', 'type': 'ColumnName'},
 {'id': 'DOUBLE', 'type': 'Data Type'},
 {'id': 'Reported revenue for the quarter.', 'type': 'Comment'},
 {'id': 'fi_net_earn', 'type': 'ColumnName'},
 {'id': 'Net earnings reported for the quarter.', 'type': 'Comment'},
 {'id': 'fi_basic_eps', 'type': 'ColumnName'},
 {'id': 'Basic earnings per share for the 

In [39]:
def restructure_and_rename_nodes(data_list):
    """
    Restructures a list of JSON-like dictionaries to group nodes by the 'node' key,
    renames 'name' to 'id', and returns a dictionary with 'nodes' as the key.

    Args:
        data_list: A list of dictionaries, where each dictionary contains a 'node' key.

    Returns:
        A dictionary where the 'nodes' key maps to a list of node dictionaries with 'id' instead of 'name'.
    """
    if not isinstance(data_list, list):
      return data_list # raise TypeError("Input must be a list")

    nodes = []
    for item in data_list:
        if isinstance(item, dict) and 'node' in item and isinstance(item['node'], dict):
            node = item['node'].copy()  # Create a copy to avoid modifying the original
            if 'name' in node:
                node['id'] = node.pop('name')  # Rename 'name' to 'id'
            nodes.append(node)

    return {"nodes": nodes}

restructure_and_rename_nodes(asd)

{'nodes': [{'type': 'TableName', 'id': 'ProspectIncremental'},
  {'type': 'SchemaName', 'id': 'wh_db_stage'},
  {'type': 'ColumnName', 'id': 'agencyid'},
  {'type': 'ColumnName', 'id': 'lastname'},
  {'type': 'ColumnName', 'id': 'firstname'},
  {'type': 'ColumnName', 'id': 'middleinitial'},
  {'type': 'ColumnName', 'id': 'gender'},
  {'type': 'ColumnName', 'id': 'addressline1'},
  {'type': 'ColumnName', 'id': 'addressline2'},
  {'type': 'ColumnName', 'id': 'postalcode'},
  {'type': 'ColumnName', 'id': 'city'},
  {'type': 'ColumnName', 'id': 'state'},
  {'type': 'ColumnName', 'id': 'country'},
  {'type': 'ColumnName', 'id': 'phone'},
  {'type': 'ColumnName', 'id': 'income'},
  {'type': 'ColumnName', 'id': 'numbercars'},
  {'type': 'ColumnName', 'id': 'numberchildren'},
  {'type': 'ColumnName', 'id': 'maritalstatus'},
  {'type': 'ColumnName', 'id': 'age'},
  {'type': 'ColumnName', 'id': 'creditrating'},
  {'type': 'ColumnName', 'id': 'ownorrentflag'},
  {'type': 'ColumnName', 'id': 'empl

In [65]:
def restructure_json_list(data_list):
    """
    Restructures a list of JSON-like dictionaries to group relationships by the 'relationship' key.

    Args:
        data_list: A list of dictionaries, where each dictionary contains a 'relationship' key.

    Returns:
        A dictionary where the 'relationship' key maps to a list of relationship dictionaries.
    """
    if not isinstance(data_list, list):
      return data_list #raise TypeError("Input must be a list")

    nodes = []
    for item in data_list:
        if isinstance(item, dict) and 'node' in item and isinstance(item['node'], dict):
            node = item['node'].copy()  # Create a copy to avoid modifying the original
            if 'name' in node:
                node['id'] = node.pop('name')  # Rename 'name' to 'id'
            nodes.append(node)

    #return {"nodes": nodes}

    relationships = []
    for item in data_list:
        if isinstance(item, dict) and 'relationship' in item and isinstance(item['relationship'], dict):
            relationships.append(item['relationship'])
        #if you have other keys in the original list, you can add more elif statements here.

    return {"nodes": nodes,
        "relationships": relationships}


restructured_data = restructure_json_list(asd2)

restructured_data


{'nodes': [{'id': 'sk_companyid', 'type': 'ColumnName'},
  {'id': 'Financial', 'type': 'TableName'},
  {'id': 'BIGINT', 'type': 'Data Type'},
  {'id': 'wh_db', 'type': 'SchemaName'},
  {'id': 'Company SK.', 'type': 'Comment'},
  {'id': 'fi_year', 'type': 'ColumnName'},
  {'id': 'INT', 'type': 'Data Type'},
  {'id': 'Year of the quarter end.', 'type': 'Comment'},
  {'id': 'fi_qtr', 'type': 'ColumnName'},
  {'id': 'Quarter number that the financial information is for: valid values 1, 2, 3, 4.',
   'type': 'Comment'},
  {'id': 'fi_qtr_start_date', 'type': 'ColumnName'},
  {'id': 'DATE', 'type': 'Data Type'},
  {'id': 'Start date of quarter.', 'type': 'Comment'},
  {'id': 'fi_revenue', 'type': 'ColumnName'},
  {'id': 'DOUBLE', 'type': 'Data Type'},
  {'id': 'Reported revenue for the quarter.', 'type': 'Comment'},
  {'id': 'fi_net_earn', 'type': 'ColumnName'},
  {'id': 'Net earnings reported for the quarter.', 'type': 'Comment'},
  {'id': 'fi_basic_eps', 'type': 'ColumnName'},
  {'id': 'Bas

In [67]:
restructured_data = restructure_json_list(asd2)

restructured_data["nodes"]

[{'id': 'sk_companyid', 'type': 'ColumnName'},
 {'id': 'Financial', 'type': 'TableName'},
 {'id': 'BIGINT', 'type': 'Data Type'},
 {'id': 'wh_db', 'type': 'SchemaName'},
 {'id': 'Company SK.', 'type': 'Comment'},
 {'id': 'fi_year', 'type': 'ColumnName'},
 {'id': 'INT', 'type': 'Data Type'},
 {'id': 'Year of the quarter end.', 'type': 'Comment'},
 {'id': 'fi_qtr', 'type': 'ColumnName'},
 {'id': 'Quarter number that the financial information is for: valid values 1, 2, 3, 4.',
  'type': 'Comment'},
 {'id': 'fi_qtr_start_date', 'type': 'ColumnName'},
 {'id': 'DATE', 'type': 'Data Type'},
 {'id': 'Start date of quarter.', 'type': 'Comment'},
 {'id': 'fi_revenue', 'type': 'ColumnName'},
 {'id': 'DOUBLE', 'type': 'Data Type'},
 {'id': 'Reported revenue for the quarter.', 'type': 'Comment'},
 {'id': 'fi_net_earn', 'type': 'ColumnName'},
 {'id': 'Net earnings reported for the quarter.', 'type': 'Comment'},
 {'id': 'fi_basic_eps', 'type': 'ColumnName'},
 {'id': 'Basic earnings per share for the 

In [7]:
for rel in json_data["relationships"]:
    source = rel["source"]
    target = rel["target"]
    relation = rel["relation"]

    query = f"""
    MATCH (s {{id: $source}}), (t {{id: $target}})
    MERGE (s)-[:{relation}]->(t)
    RETURN type(r) AS relation;
    """

    print(query)


    MATCH (s {id: $source}), (t {id: $target})
    MERGE (s)-[:PART_OF]->(t)
    RETURN type(r) AS relation;
    

    MATCH (s {id: $source}), (t {id: $target})
    MERGE (s)-[:PART_OF]->(t)
    RETURN type(r) AS relation;
    

    MATCH (s {id: $source}), (t {id: $target})
    MERGE (s)-[:TYPE_OF]->(t)
    RETURN type(r) AS relation;
    

    MATCH (s {id: $source}), (t {id: $target})
    MERGE (s)-[:PART_OF]->(t)
    RETURN type(r) AS relation;
    

    MATCH (s {id: $source}), (t {id: $target})
    MERGE (s)-[:TYPE_OF]->(t)
    RETURN type(r) AS relation;
    

    MATCH (s {id: $source}), (t {id: $target})
    MERGE (s)-[:PART_OF]->(t)
    RETURN type(r) AS relation;
    

    MATCH (s {id: $source}), (t {id: $target})
    MERGE (s)-[:TYPE_OF]->(t)
    RETURN type(r) AS relation;
    

    MATCH (s {id: $source}), (t {id: $target})
    MERGE (s)-[:PART_OF]->(t)
    RETURN type(r) AS relation;
    

    MATCH (s {id: $source}), (t {id: $target})
    MERGE (s)-[:TYPE_OF]->(t)
  

In [45]:

import re
import json
import pandas as pd

def summarize_lineage_issues(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

    # Split by script blocks: script_name followed by JSON object
    blocks = re.findall(r'([^\n]+\.sql):\s*\{(.*?)\}(?=\s*[^\n]+\.sql:|\Z)', content, re.DOTALL)

    summary = []
    total_columns = 0
    total_ok_columns = 0

    for script_name, json_block in blocks:
        # Wrap back to full JSON format
        json_str = "{" + json_block.strip() + "}"
        try:
            data = json.loads(json_str)
        except json.JSONDecodeError as e:
            summary.append({
                "script": script_name,
                "errors": [f"JSON parse error: {e}"],
                "error_count": 1,
                "total_columns": 0,
                "ok_columns": 0
            })
            continue

        column_count = len(data)
        error_columns = [col for col, issues in data.items() if issues]
        ok_columns = column_count - len(error_columns)

        total_columns += column_count
        total_ok_columns += ok_columns

        summary.append({
            "script": script_name,
            "errors": error_columns,
            "error_count": len(error_columns),
            "total_columns": column_count,
            "ok_columns": ok_columns
        })

    # Print table
    print(f"{'Script':<35} | {'Errors'}")
    print("-" * 80)
    for entry in summary:
        if entry["error_count"] == 0:
            status = "✅ OK"
        elif "JSON parse error" in entry["errors"][0]:
            status = f"❌ {entry['errors'][0]}"
        else:
            status = f"❌ {entry['error_count']} issue(s): {', '.join(entry['errors'])}"
        print(f"{entry['script']:<35} | {status}")

    # Print totals
    print("\n--- Summary ---")
    print(f"Total scripts reviewed:   {len(summary)}")
    print(f"Total columns reviewed:   {total_columns}")
    print(f"Total columns without issues: {total_ok_columns}")
    print(f"Total columns with issues:    {total_columns - total_ok_columns}")

    
    return summary

def extract_detailed_issues(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

    blocks = re.findall(r'([^\n]+\.sql):\s*\{(.*?)\}(?=\s*[^\n]+\.sql:|\Z)', content, re.DOTALL)

    detailed_issues = []

    for script_name, json_block in blocks:
        json_str = "{" + json_block.strip() + "}"
        try:
            data = json.loads(json_str)
        except json.JSONDecodeError as e:
            detailed_issues.append({
                "script": script_name,
                "column": None,
                "issue_description": f"JSON parse error: {e}"
            })
            continue

        # Extracting column-level issues
        for column, issues in data.items():
            if issues:  # assuming 'issues' is a list of descriptions
                for issue_desc in issues:
                    detailed_issues.append({
                        "script": script_name,
                        "column": column,
                        "issue_description": issue_desc
                    })

    return pd.DataFrame(detailed_issues)


file_path = "C:\lopu-kg-test\project\EVALUATOR_OUTPUT.txt"

sum = summarize_lineage_issues(file_path)

df = pd.DataFrame(sum)

print(df)


detail_issues = extract_detailed_issues(file_path)

print(detail_issues)

Script                              | Errors
--------------------------------------------------------------------------------
1_temp_broker.sql                   | ✅ OK
1_wh_db.DimAccount.sql              | ✅ OK
1_wh_db.DimCompany.sql              | ❌ 1 issue(s): batchid
1_wh_db.DimCustomer.sql             | ❌ 1 issue(s): batchid
1_wh_db.DimDate.sql                 | ✅ OK
1_wh_db.DimSecurity.sql             | ❌ 3 issue(s): status, effectivedate, enddate
1_wh_db.DimTime.sql                 | ✅ OK
1_wh_db.DimTrade.sql                | ✅ OK
1_wh_db.FactCashBalances.sql        | ✅ OK
1_wh_db.FactHoldings.sql            | ✅ OK
1_wh_db.FactWatches.sql             | ✅ OK
1_wh_db.Financial.sql               | ✅ OK
1_wh_db.Prospect.sql                | ✅ OK
1_wh_db.StatusType.sql              | ✅ OK
1_wh_db.TaxRate.sql                 | ✅ OK
1_wh_db.TradeType.sql               | ✅ OK
1_wh_db_stage.CustomerMgmt.sql      | ✅ OK
1_wh_db_stage.FinWire.sql           | ✅ OK
1_wh_db_stage.ProspectIncr

In [2]:
df.style.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('color', '#333'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('padding', '8px'), ('border', '1px solid #ccc')]},
]).set_properties(**{'text-align': 'center', 'font-family': 'Arial'})

,script,errors,error_count,total_columns,ok_columns
0,1_temp_broker.sql,[],0,9,9
1,1_wh_db.DimAccount.sql,[],0,7,7
2,1_wh_db.DimCompany.sql,['batchid'],1,20,19
3,1_wh_db.DimCustomer.sql,['batchid'],1,24,23
4,1_wh_db.DimDate.sql,[],0,18,18
5,1_wh_db.DimSecurity.sql,"['status', 'effectivedate', 'enddate']",3,14,11
6,1_wh_db.DimTime.sql,[],0,10,10
7,1_wh_db.DimTrade.sql,[],0,7,7
8,1_wh_db.FactCashBalances.sql,[],0,4,4
9,1_wh_db.FactHoldings.sql,[],0,11,11


In [48]:
import pandas as pd
import numpy as np

def summarize_data(dataframe):
    """
    Calculates summary statistics for numeric columns in a DataFrame and
    adds a summary row with a custom index.  Non-numeric columns are
    handled by either summing (if it makes sense) or filling with a string.

    Args:
        dataframe (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A new DataFrame with the summary row appended.
    """
    # Calculate summary for numeric columns
    numeric_df = dataframe.select_dtypes(include=np.number)
    if not numeric_df.empty:
        summary_data = numeric_df.agg(['sum', 'mean'])
        summary_row_values = summary_data.iloc[0].to_dict()  # Use the first row of agg
    else:
        summary_row_values = {}

    # Handle non-numeric columns
    for col in dataframe.columns:
        if col not in summary_row_values:
            # Try summing non-numeric columns, if it fails, fill with a string
            try:
                # Convert lists to strings for summation
                if isinstance(dataframe[col][0], list):
                    summary_row_values[col] = ", ".join(map(str, sum(dataframe[col], [])))
                else:
                    summary_row_values[col] = dataframe[col].astype(str).sum()
            except TypeError:
                summary_row_values[col] = ''

    # Create a Series from the summary values
    summary_row = pd.Series(summary_row_values, name='Summary')

    # Concatenate the original DataFrame and the summary row (as a DataFrame)
    df_with_summary = pd.concat([dataframe, summary_row.to_frame().T], ignore_index=True)

    # Round all numeric values in the entire DataFrame
    for col in df_with_summary.columns:
        if pd.api.types.is_numeric_dtype(df_with_summary[col]):
            df_with_summary[col] = df_with_summary[col].apply(lambda x: round(x) if pd.notnull(x) else x)


    # Round numeric values in the summary row and set 'script' to 'Summary'
    summary_row = df_with_summary.iloc[-1] # Get the last row
    for col, value in summary_row.items():
        if isinstance(value, (int, float)):
            summary_row[col] = round(value)
    summary_row['script'] = 'Summary'
    df_with_summary.iloc[-1] = summary_row # set the last row


    # Calculate accuracy percentage
    if 'error_count' in df_with_summary.columns and 'total_columns' in df_with_summary.columns:
        df_with_summary['accuracy'] = ((df_with_summary['total_columns'] - df_with_summary['error_count']) / df_with_summary['total_columns']) * 100
    else:
        df_with_summary['accuracy'] = 'N/A'  # Or some other default value
    
    # Round the accuracy column
    if 'accuracy' in df_with_summary.columns:
        df_with_summary['accuracy'] = df_with_summary['accuracy'].apply(lambda x: round(x,1) if pd.notnull(x) and x != 'N/A' else x)

    return df_with_summary


asd = summarize_data(df)
asd = asd.drop('errors', axis=1)


C:\Users\kasutaja\AppData\Local\Temp\ipykernel_21784\644332788.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_row[col] = round(value)
C:\Users\kasutaja\AppData\Local\Temp\ipykernel_21784\644332788.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_row['script'] = 'Summary'


In [49]:
def make_last_row_bold(df):
    """
    Makes the last row of a pandas DataFrame bold in the HTML representation.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        str: HTML representation of the DataFrame with the last row in bold.
    """
    def style_last_row(row):
        if row.name == df.index[-1]:
            return ['font-weight: bold'] * len(row)
        return [''] * len(row)

    return df.style.apply(style_last_row, axis=1).to_html()

asd = asd.astype(str)

from IPython.display import HTML
HTML(make_last_row_bold(asd))




,script,error_count,total_columns,ok_columns,accuracy
0,1_temp_broker.sql,0,9,9,100.0
1,1_wh_db.DimAccount.sql,0,7,7,100.0
2,1_wh_db.DimCompany.sql,1,20,19,95.0
3,1_wh_db.DimCustomer.sql,1,24,23,95.8
4,1_wh_db.DimDate.sql,0,18,18,100.0
5,1_wh_db.DimSecurity.sql,3,14,11,78.6
6,1_wh_db.DimTime.sql,0,10,10,100.0
7,1_wh_db.DimTrade.sql,0,7,7,100.0
8,1_wh_db.FactCashBalances.sql,0,4,4,100.0
9,1_wh_db.FactHoldings.sql,0,11,11,100.0


In [36]:
import pandas as pd
import numpy as np

def summarize_data(dataframe):
    """
    Calculates summary statistics for numeric columns in a DataFrame and
    adds a summary row with a custom index. Non-numeric columns are
    handled by either summing (if it makes sense) or filling with a string.

    Args:
        dataframe (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A new DataFrame with the summary row appended.
    """
    # Calculate summary for numeric columns
    numeric_df = dataframe.select_dtypes(include=np.number)
    if not numeric_df.empty:
        summary_data = numeric_df.agg(['sum', 'mean'])
        summary_row_values = summary_data.iloc[0].to_dict()  # Use the first row of agg
    else:
        summary_row_values = {}

    # Handle non-numeric columns
    for col in dataframe.columns:
        if col not in summary_row_values:
            # Try summing non-numeric columns, if it fails, fill with a string
            try:
                # Convert lists to strings for summation
                if isinstance(dataframe[col][0], list):
                    summary_row_values[col] = ", ".join(map(str, sum(dataframe[col], [])))
                else:
                    summary_row_values[col] = dataframe[col].astype(str).sum()
            except TypeError:
                summary_row_values[col] = ''

    # Create a Series from the summary values
    summary_row = pd.Series(summary_row_values, name='Summary')

    # Concatenate the original DataFrame and the summary row (as a DataFrame)
    df_with_summary = pd.concat([dataframe, summary_row.to_frame().T], ignore_index=True)

    # Round all numeric values in the entire DataFrame
    for col in df_with_summary.columns:
        if pd.api.types.is_numeric_dtype(df_with_summary[col]):
            df_with_summary[col] = df_with_summary[col].apply(lambda x: round(x, 2) if pd.notnull(x) else x) #<---  Round to 2 decimal places

    # Round numeric values in the summary row and set 'script' to 'Summary'
    summary_row = df_with_summary.iloc[-1]  # Get the last row
    for col, value in summary_row.items():
        if isinstance(value, (int, float)):
            summary_row[col] = round(value, 2) #<--- Round to 2 decimal places
    summary_row['script'] = 'Summary'
    df_with_summary.iloc[-1] = summary_row  # set the last row

    # Calculate accuracy percentage
    if 'error_count' in df_with_summary.columns and 'total_columns' in df_with_summary.columns:
        df_with_summary['accuracy'] = ((df_with_summary['total_columns'] - df_with_summary['error_count']) / df_with_summary['total_columns']) * 100
    else:
        df_with_summary['accuracy'] = 'N/A'  # Or some other default value
    
    # Round the accuracy column
    if 'accuracy' in df_with_summary.columns:
        df_with_summary['accuracy'] = df_with_summary['accuracy'].apply(lambda x: round(x, 2) if pd.notnull(x) and x != 'N/A' else x) # Round to 2 decimal places

    df_with_summary['accuracy'] = round(df_with_summary['accuracy'], 1)

    return df_with_summary


def make_last_row_bold(df):
    """
    Makes the last row of a pandas DataFrame bold in the HTML representation.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        str: HTML representation of the DataFrame with the last row in bold.
    """
    def style_last_row(row):
        if row.name == df.index[-1]:
            return ['font-weight: bold'] * len(row)
        return [''] * len(row)

    return df.style.apply(style_last_row, axis=1).to_html() #<---- .to_html() added here



# Sample DataFrame (for testing)
data = {
    'script': ['a', 'b', 'c', 'd'],
    'error_count': [1, 2, 0, 5],
    'total_columns': [10, 10, 10, 10],
    'some_value': [1.2345, 2.3456, 3.4567, 4.5678],
    'accuracy': [99.234, 88.423, 76.123, 92.231]
}
df = pd.DataFrame(data)

# Apply the summarize_data function
df_result = summarize_data(df)

# Drop the 'error_count' column
df_dropped_single = df_result.drop('error_count', axis=1, errors='ignore')  # Use errors='ignore'

# Print the result
print(df_dropped_single)


from IPython.display import HTML
HTML(make_last_row_bold(df_dropped_single))


    script total_columns some_value  accuracy
0        a            10     1.2345      90.0
1        b            10     2.3456      80.0
2        c            10     3.4567     100.0
3        d            10     4.5678      50.0
4  Summary          40.0       11.6      80.0


C:\Users\kasutaja\AppData\Local\Temp\ipykernel_21784\4266169823.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_row[col] = round(value, 2) #<--- Round to 2 decimal places
C:\Users\kasutaja\AppData\Local\Temp\ipykernel_21784\4266169823.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_row['script'] = 'Summary'


,script,total_columns,some_value,accuracy
0,a,10,1.234500,90.000000
1,b,10,2.345600,80.000000
2,c,10,3.456700,100.000000
3,d,10,4.567800,50.000000
4,Summary,40.000000,11.600000,80.000000


In [ ]:
detail_issues = detail_issues.set_table_styles([
    {'selector': 'th', 'props': [('width', '200px'), ('text-align', 'left')]},  # Header styles
    {'selector': 'td', 'props': [('width', '200px'), ('text-align', 'left'), ('word-wrap', 'break-word')]}  # Cell styles
])

df.style.apply(style_last_row, axis=1)

from IPython.display import HTML
HTML(detail_issues.to_html())

AttributeError: 'DataFrame' object has no attribute 'set_table_styles'

In [65]:
import pandas as pd
from IPython.display import HTML

def style_dataframe(df, column_width="200px"):
    """
    Styles a Pandas DataFrame for HTML output, applying text wrapping.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_width (str, optional):  The width of the columns. Defaults to "200px".
            Use "auto" to allow content to determine width.

    Returns:
        str:  HTML representation of the styled DataFrame.
    """

    # Apply the styles.  Use a list to make it clear we're adding multiple styles.
    styled_df = df.style.set_table_styles([
        {'selector': 'th',
         'props': [('width', column_width),
                   ('text-align', 'right')]},
        {'selector': 'td',
         'props': [('width', column_width),
                   ('text-align', 'right'),
                   ('word-wrap', 'break-word')]}
    ])


    
    return HTML(styled_df.to_html(show_index=False))  # Return the HTML string



style_dataframe(detail_issues.head(5))

,script,column,issue_description
0,1_wh_db.DimCompany.sql,batchid,batchid is hardcoded as 1 in SQL but not explicitly documented in the extracted data model.
1,1_wh_db.DimCustomer.sql,batchid,Hardcoded value '1' assigned
2,1_wh_db.DimSecurity.sql,status,"Transformation logic in SQL includes a CASE mapping from trimmed substring to descriptive status values (e.g., 'ACTV' -> 'Active'), but extracted model shows only raw substring extraction without CASE mapping."
3,1_wh_db.DimSecurity.sql,effectivedate,"Extracted model lists effectivedate as directly from recdate, but SQL applies conditional logic to choose the later date between SEC.effectivedate and dc.EffectiveDate."
4,1_wh_db.DimSecurity.sql,enddate,"Extracted model shows enddate derived from LEAD window function with fallback date, but SQL further adjusts enddate by bounding it with dc.EndDate in final SELECT."
